# How to read MSG SEVIRI data from the TROPOS archive?

This tutorial shows how to read MSG SEVIRI data using the MSevi data container class which is part of the python library collection tropy. The MSevi data container load sat.-data from hdf file without restrictions if they hdf is available and the region is inside the predefined 'eu'-domain. If not, then the MSevi class can also load the original HRIT files, but in this case no HRV input is implemented.

## Import Libraries

In [ ]:
%matplotlib inline

import pylab as pl
import numpy as np
import datetime
from l15_msevi.msevi import MSevi


SEVIRI data are loaded into the MSevidata container. 

## Configuration

For configuration, we have to set region, scan_type and time (as object).

In [ ]:
time = datetime.datetime( 2013, 6, 8, 12, 0)
region = 'eu'
scan_type = 'rss'

Initialize the Data Container.

In [ ]:
s = MSevi(time = time, region = region, scan_type = scan_type)

## Load Infrared Channel

Start with the infrared channel at 10.8 um.

In [ ]:
s.load('IR_108')

Now, the sat data are available as radiance (i.e. already converted from the counts using slope and offset). A dictionary s.rad is used to store the radiance information.

In [ ]:
print (s.rad)

What you want for infrared channel is perhaps the brightness temperature (BT). To get it, the is the method rad2bt that creates a BT dictionary under s.bt

In [ ]:
s.rad2bt('IR_108')
pl.imshow(s.bt['IR_108'])

# Adjusting Region Configuration

In [ ]:
region = ((216, 456), (1676, 2076))
s = MSevi(time = time, region = region, scan_type = scan_type)

In [ ]:
s.load('IR_108')
s.rad2bt('IR_108')
pl.imshow(s.bt['IR_108'])

# Getting geo-reference

The MSevi class also provides the method to read geo-reference, i.e. longitude and latitude, for the selected region cutout. The method is called s.lonlat() and the result is stored in the attributes s.lon and s.lat

In [ ]:
s.lonlat()
print s.lon, s.lat

Now, we plot again, but geo-referenced.

In [ ]:
pl.pcolormesh(s.lon, s.lat, s.bt['IR_108'])

# Loading several channels

Now, we make a channel list and read all channels. The easied way is the get the default channel list from the msevi config. 

In [ ]:
from l15_msevi.msevi_config import _narrow_channels
print _narrow_channels

full_channel_list = _narrow_channels + ['HRV',]

In [ ]:
s.load(full_channel_list)
print( s.rad.keys())

Juchu! All channels are loaded! 

Now, we convert all IR channel radiances to BTs. No arg means all...

In [ ]:
s.rad2bt()
print (s.bt.keys())

Time to explore the content. We loop over all IR channels...

In [ ]:
for chan_name in s.bt.keys():
    pl.figure()
    pl.pcolormesh(s.lon, s.lat, s.bt[chan_name])
    pl.title(chan_name)

Wow, what a welth of information ;-) 

But, visible is still missing, right? For this, there is the second conversion method called s.rad2refl that calculated reflectances. Note, no cos of solar zenith angle correction is applied here!

In [ ]:
s.rad2refl()
print (s.ref.keys())

Now, we plot the visible stuff.

First, the narrow-band channels. 

In [ ]:
for chan_name in ['VIS006', 'VIS008', 'IR_016']:
    pl.figure()
    pl.pcolormesh(s.lon, s.lat, s.ref[chan_name], cmap = pl.cm.gray)
    pl.title(chan_name)

Sat-Images are really beautiful!

# HRV channel

We already loaded did all the input for HRV. We 
* loaded the radiances of the HRV channel
* converted them to reflectances
* and loaded georeference

With the narrow-band geo-ref, also a high-res geo-ref was calculated (just by linear interpolation). The high-res. lon and lat values are stored in the attributes s.hlon and s.hlat and can be used to plot and analyze HRV data

In [ ]:
pl.pcolormesh(s.hlon, s.hlat, s.ref['HRV'], cmap = pl.cm.gray)
pl.title('HRV')

Great detail in there!